## Sequence Embeddings

The total number of records in the dataset is close to one million, and
there are 0.1 million unique users. The time spent by each user on each of
the web pages is also tracked along with the final status if the user bought
the product or not.

In [4]:
import os
os.environ["PYSPARK_SUBMIT_ARGS"] = "--master local[*] pyspark-shell"

import findspark
findspark.init()

import pyspark
#import SparkSession
from pyspark.sql import SparkSession

In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [6]:
from pyspark.ml.feature import StringIndexer
from pyspark.sql.window import Window

# scikit-learn
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier
from statsmodels.api import Logit
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
%matplotlib inline

# others
import pandas as pd
import numpy as np
import sys
import itertools
import re
from random import sample
import time

In [7]:
# !pip install gensim

In [8]:
from gensim.models.doc2vec import LabeledSentence
from gensim.models import Doc2Vec
from gensim.models import Word2Vec

In [9]:
#create SparkSession
spark=SparkSession.builder.appName('seq_embedding').getOrCreate()

In [10]:
#reading a file
df = spark.read.csv('embedding_dataset.csv',header=True,inferSchema=True)

AnalysisException: 'Path does not exist: file:/C:/Users/ctw00071/Desktop/Desktop/PySpark/chapter_9_NLP/embedding_dataset.csv;'

In [ ]:
df.count()

In [ ]:
df.printSchema()

In [ ]:
df.select('user_id').distinct().count()

In [ ]:
df.groupBy('page').count().orderBy('count',ascending=False).show(10,False)

In [ ]:
df.select(['user_id','page','visit_number','time_spent','converted']).show(10,False)

The whole idea of sequence embeddings is **to translate the series
of steps taken by the user during his or her online journey into a page
sequence, which can be used for calculating embedding scores.** The
first step is to remove any of the consecutive duplicate pages during
the journey of a user. We create an additional column that captures the previous page of a user. **Window** is a function in spark that helps to apply
certain logic specific to individual or group of rows in the dataset.

In [ ]:
# window for each user order by timestamp
w = Window.partitionBy("user_id").orderBy('timestamp')

In [ ]:
#creating a lagged column 
df = df.withColumn("previous_page", lag("page", 1, 'started').over(w))

In [ ]:
df.select('user_id','timestamp','previous_page','page').show(10,False)

Now, we create a function to check if the current page is similar to the
previous page and indicate the same in a new column indicator. Indicator
cumulative is the column to track the number of distinct pages during the
user's journey.

In [ ]:
# adding an indicator if current page is same as next page
def indicator(page, prev_page):
    if page == prev_page:
        return 0
    else:
        return 1
    
page_udf = udf(indicator,IntegerType())

In [ ]:
# adding a column for indicator and cumulative indicator
df = df.withColumn("indicator",page_udf(col('page'),col('previous_page'))) \
        .withColumn('indicator_cummulative',sum(col('indicator')).over(w))

In [ ]:
df.select('previous_page','page','indicator','indicator_cummulative').show(20,False)

In [ ]:
# create window with user and indicator cummulative
w2 = Window.partitionBy(["user_id",'indicator_cummulative']).orderBy('timestamp')

In [ ]:
# adding a column with time spent cumulative ( time spent by a user on a page  visited in continuation )
df = df.withColumn('time_spent_cummulative',sum(col('time_spent')).over(w2))

In [ ]:
df.select('timestamp','previous_page','page','indicator','indicator_cummulative','time_spent','time_spent_cummulative').show(20,False)

In the next stage, we calculate the aggregated time spent on similar
pages so that only a single record can be kept for representing consecutive
pages.

In [ ]:
# creating a window to get final page and final timespent 
w3 = Window.partitionBy(["user_id",'indicator_cummulative']).orderBy(col('timestamp').desc())

In [ ]:
# Add column for final page category and final time spent
df = df.withColumn('final_page',first('page').over(w3))\
     .withColumn('final_time_spent',first('time_spent_cummulative').over(w3))


In [ ]:
df.select(['time_spent_cummulative','indicator_cummulative','page','final_page','final_time_spent']).show(10,False)

In [ ]:
# user and pagelevel aggregation  
aggregations=[]
aggregations.append(max(col('final_page')).alias('page_emb'))
aggregations.append(max(col('final_time_spent')).alias('time_spent_emb'))
aggregations.append(max(col('converted')).alias('converted_emb'))

In [ ]:
#selecting relevant columns
# extracting the dataframe with the data frame that will be used for embedding
df_embedding = df.select(['user_id','indicator_cummulative','final_page','final_time_spent','converted']).groupBy(['user_id','indicator_cummulative']).agg(*aggregations)

In [ ]:
df_embedding.count()

In [ ]:
df_embedding.show(3, False)

Finally, we use a collect list to combine all the pages of a user's journey
into a single list and for time spent as well. As a result, we end with the user
journey in the form of a page list and time spent list.

In [ ]:
# create a partition by user id ordered by indicator cumulative to get the journey
w4 = Window.partitionBy(["user_id"]).orderBy('indicator_cummulative')
w5 = Window.partitionBy(["user_id"]).orderBy(col('indicator_cummulative').desc())

In [ ]:
df_embedding = df_embedding.withColumn('journey_page', collect_list(col('page_emb')).over(w4))\
                          .withColumn('journey_time_temp', collect_list(col('time_spent_emb')).over(w4)) \
                         .withColumn('journey_page_final',first('journey_page').over(w5))\
                        .withColumn('journey_time_final',first('journey_time_temp').over(w5)) \
                        .select(['user_id','journey_page_final','journey_time_final','converted_emb'])

In [ ]:
df_embedding.select('user_id','journey_page_final','journey_time_final').show(10)

In [ ]:
df_embedding.count()

In [ ]:
df_embedding.select('user_id').distinct().count()

We continue with only unique user journeys. Each user is represented
by a single journey and time spent vector.

In [ ]:
df_embedding = df_embedding.dropDuplicates()

In [ ]:
df_embedding.count()

In [ ]:
df_embedding.select('user_id').distinct().count()

In [ ]:
df_embedding.select('user_id','journey_page_final','journey_time_final').show(10)

Now that we have the user journeys and time spent list, we convert this
dataframe to a Pandas dataframe and build a **word2vec** model using these
journey sequences. We have **to install a gensim library** first in order to use
word2vec.We use the embedding size of 100 to keep it simple.

In [ ]:
# create pandas dataframe for embedding
pd_df_embedding = df_embedding.toPandas()

In [ ]:
pd_df_embedding.head(5)

In [ ]:
# making sure we don't have journeys with length less than 4
pd_df_embedding = pd_df_embedding[pd_df_embedding['journey_length'] > 4 ]

In [ ]:
# reset index
pd_df_embedding = pd_df_embedding.reset_index(drop=True)

In [ ]:
# train model
EMBEDDING_SIZE = 100
model = Word2Vec(pd_df_embedding['journey_page_final'], size = EMBEDDING_SIZE)

In [ ]:
model.total_train_time

As we can observe, the vocabulary size is 7 because we were dealing
with 7 page categories only. Each of these pages category now can be
represented with help of an embedding vector of size 100.

In [ ]:
# summarize the loaded model
print(model)

In [ ]:
# summarize vocabulary
page_categories = list(model.wv.vocab)

In [ ]:
# page categories 
print(page_categories)

In [ ]:
# sample embedding
print(model['reviews'])

In [ ]:
# embedding shape 
model['offers'].shape

To create the **embedding matrix**, we can use a model and pass the
model vocabulary; it would result in a matrix of size (7,100.)

In [ ]:
# capturing embedding matrix
X = model[model.wv.vocab]

In [ ]:
# embedding matrix shapee
X.shape

In order to better understand the relation between these page
categories, we can use a dimensionality reduction technique (PCA) and
plot these seven page embeddings on a two-dimensional space.

In [ ]:
# run PCA with 2 compopnent to visualize page category embedding
pca = PCA(n_components=2)
result = pca.fit_transform(X)

In [ ]:
# plotting with page-categories
# create a scatter plot of the projection
plt.figure(figsize=(10,10))
plt.scatter(result[:, 0], result[:, 1])

for i,page_category in enumerate(page_categories):
    plt.annotate(page_category,horizontalalignment='right', verticalalignment='top',xy=(result[i, 0], result[i, 1]))

plt.show()

As we can clearly see, the embeddings of **buy** and **added to cart** are
near to each other in terms of similarity whereas **homepage** and **product
info** are also closer to each other. **Offers** and **reviews** are totally separate
when it comes to representation through embeddings. **These individual
embeddings can be combined and used for user journey comparison and
classification using Machine Learning.**